# 单变量线性回归 (Linear Regression with One Variable)

> **导语**：线性模型是机器学习的基石，几乎所有复杂的深度学习网络都是从这个基础模型演化而来的。我们将从最基础的线性回归开始学起。

## 1. 场景还原：从“人工直觉”到“机器拟合”

想象一下，你现在是美国波特兰市的一名房产中介。你手头有一张表格，记录了最近成交的房屋数据。

**任务 ($T$)**：当客户咨询一套 **3000 平方英尺** 的新房子能卖多少钱时，你要给出一个科学的估价。

### 1.1 数据可视化 (Visualization)

人类的大脑不擅长直接在表格中寻找规律，但对图形极其敏感。我们首先将数据映射到坐标系中，绘制成**散点图 (Scatter Plot)**：

- **横轴 ($x$)**：房屋面积 (Size)
    
- **纵轴 ($y$)**：房屋价格 (Price)
    

### 1.2 物理建模 (Physical Modeling)

ok,接下来我们要做的就是找到一个函数，计算面积和价格关系，假设函数是：
$$ h_{w,b}(x) = w \cdot x + b$$
我们要做的就是找到w和b的最佳值，使得函数可以尽可能多地穿过这些数据点。

1. **收集数据**：在坐标轴上标出所有的点（构建**训练集 Training Set**）。
    
2. **初步摆放**：你拿出一把**透明直尺**，将尺子随机放在图纸上（**随机初始化参数 Random Initialization**）。
    
3. **视觉评估**：发现大部分点都在尺子上方，偏差较大（**计算误差 Error**）。
    
4. **人工调整**：将尺子整体上移，并增大倾斜角度（**更新模型参数 Update Parameters**）。
    
5. **最终定型**：找到最贴合数据点的位置，画出红线（**模型收敛 Model Convergence**）。
    

**恭喜你！这 5 个步骤，完美映射了机器学习的完整训练生命周期。*

---

## 2.程序实现: 场景分析

### 2.1初始化

程序实现其实就是把物理建模的过程用代码实现一遍。其中收集数据这一步是最简单的，只需要绘制散点图即可。接下来我们进行初步摆放：假设机器进行了第一次尝试（瞎猜），它把尺子摆在了 w = 1.0, b = 1.0 的位置。此时预测函数为：$h_{w,b}(x) = 1.0 \cdot x + 1.0$
为了直观地看到这个函数的效果，我们用 Python 将**真实数据点**、**预测函数线**，以及它们之间的**差距（误差项）**画出来：


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1. 收集数据 (真实的点)
x_train = np.array([1.0, 1.5, 2.0, 2.5, 3.0])
y_train = np.array([1.5, 2.5, 3.5, 4.0, 5.0])

# 2. 模拟的函数图像 (w=1.0, b=1.0)
def h(x, w, b):
    return w * x + b

w_guess, b_guess = 1.0, 1.0
x_line = np.linspace(0.5, 3.5, 100)
y_line = h(x_line, w_guess, b_guess)

# 绘图
plt.figure(figsize=(8, 5))
plt.scatter(x_train, y_train, color='black', marker='x', s=100, label='Real Data (y)') # 画样本点
plt.plot(x_line, y_line, color='blue', label='Prediction Line (h(x))') # 画预测线

# 3. 描绘差距 (误差竖线)
for i in range(len(x_train)):
    y_pred = h(x_train[i], w_guess, b_guess)
    # 画一条虚线连接真实点和预测线，这就是“差距”
    plt.plot([x_train[i], x_train[i]], [y_train[i], y_pred], color='red', linestyle='--')

plt.title("Visualizing the Gap (Error Residuals)")
plt.xlabel("Size in 1000 sqft (x)")
plt.ylabel("Price in $100k (y)")
plt.legend()
plt.grid(True)
plt.show()


> **逻辑映射**：图中连接真实点与预测线的**红色虚线**，即为模型在单个样本上的**误差 (Error)**。

我们将这 5 个点的真实值 $y^{(i)}$、预测值 $\hat{y}^{(i)}$ 以及误差，以结构化表格呈现：

|**样本索引 (i)**|**面积 x(i)**|**真实价格 y(i)**|**预测价格 y^​=1.0⋅x+1.0**|**单点误差 (y(i)−y^​(i))**|
|---|---|---|---|---|
|1|1.0|1.5|2.0|**-0.5** (预测偏高)|
|2|1.5|2.5|2.5|**0.0** (完美拟合)|
|3|2.0|3.5|3.0|**0.5** (预测偏低)|
|4|2.5|4.0|3.5|**0.5** (预测偏低)|
|5|3.0|5.0|4.0|**1.0** (预测偏低)|

### 2.3确立代价函数 (Cost Function)

我们需要一个聚合公式，将所有单点误差综合起来，给当前的 $w$ 和 $b$ 打一个“总评分”（对应性能指标 $P$：评分越低，模型越好）。

- ❌ **错误方案 1：直接求和**
    
    若直接相加，样本 1 (误差 -0.5) 和样本 3 (误差 0.5) 会发生正负抵消。总误差显示为 0，机器会误判模型完美无缺。
    
- ❌ **错误方案 2：绝对值求和 $|y - \hat{y}|$**
    
    绝对值能解决抵消问题，但其函数图像在底部呈尖锐的 “V” 字型折角。在微积分范畴中，**尖角处不可导**，这会导致后续依赖导数的优化算法卡死。
    
- ✅ **工业界标准：均方误差 (Mean Squared Error, MSE)**
    
    将每个误差先**平方**，再求平均。平方操作不仅消除了负数，还赋予了代价函数平滑的 “U” 型碗状结构（完美可导）。同时，平方会指数级放大严重的预测偏差，从而狠狠“惩罚”劣质模型。
    

由此得出线性回归中最著名的**均方误差代价函数**：

$$J(w,b) = \frac{1}{2m} \sum_{i=1}^{m} \left( h_{w,b}(x^{(i)}) - y^{(i)} \right)^2$$

> _(注：公式中除以 $2m$ 而非 $m$，是工程上的巧妙处理。后续对该平方函数求导时，指数 $2$ 落下与分母的 $2$ 抵消，使得梯度公式更简洁。)_

---

## 3. 参数寻优：梯度下降 (Gradient Descent)

目前的任务是寻找一组最佳的 $w$ 和 $b$，使得代价函数 $J(w,b)$ 的值降至最低。在微积分中，相当于找到导函数为0的点。

### 3.1 视觉化误差曲面：三维“碗”状图

由于现在我们要解决的问题变成了：寻找一组最佳的 $w$ 和 $b$，为了便于理解，我们可以直接将函数图像绘制出来，也就是将视角从二维的“面积-价格图”切换至三维的**“参数-代价图”**。

通过计算所有可能的 $(w, b)$ 组合对应的总误差 $J$，我们可以绘制出代价函数的真实面貌。

Python


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1. 准备训练数据
x_train = np.array([1.0, 1.5, 2.0, 2.5, 3.0])
y_train = np.array([1.5, 2.5, 3.5, 4.0, 5.0])
m = len(x_train)

# 2. 生成网格化参数矩阵
w_vals = np.linspace(0, 3, 100)
b_vals = np.linspace(-1, 2, 100)
W, B = np.meshgrid(w_vals, b_vals)
J_vals = np.zeros_like(W)

# 3. 计算所有参数组合的代价 J
for i in range(len(w_vals)):
    for j in range(len(b_vals)):
        error = (w_vals[i] * x_train + b_vals[j]) - y_train
        J_vals[j, i] = (1 / (2 * m)) * np.sum(error ** 2)

# 4. 绘制三维曲面与等高线图
fig = plt.figure(figsize=(12, 5))

# 左侧：3D 曲面图 (3D Surface)
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(W, B, J_vals, cmap='viridis', alpha=0.8)
ax1.set_title('3D Cost Function J(w,b)')
ax1.set_xlabel('Weight (w)')
ax1.set_ylabel('Bias (b)')
ax1.set_zlabel('Cost (J)')

# 右侧：等高线图 (Contour Plot)
ax2 = fig.add_subplot(122)
contour = ax2.contour(W, B, J_vals, levels=20, cmap='viridis')
ax2.set_title('Contour Plot (Top-down view)')
ax2.set_xlabel('Weight (w)')
ax2.set_ylabel('Bias (b)')

plt.show()


**图形解析：**

- **左图（3D 曲面）**：由于采用平方误差，这是一个平滑的“碗”状凸函数 (Convex Function)。碗底坐标即为全局最优参数。
    
- **右图（等高线图）**：这是俯视图，每一圈椭圆代表相同的误差值。靶心位置即为误差极小值点。
    

### 3.2 下山策略：梯度下降

在图像中，我们要找的就是函数值最小的点，对应的就是函数图像的谷底。

**物理直觉映射：**

想象你被蒙住双眼，空投到了等高线图最外圈的某个坐标点。你要如何走到谷底？

1. **探寻坡度**：原地用脚试探四周，感受最陡峭的下坡方向（数学映射：**对代价函数计算偏导数/梯度**）。
    
2. **迈出步伐**：沿着最陡的方向向下走一步（因为下降最快）。
    
3. **循环迭代**：不断重复“试探-迈步”，直到走到四周皆平坦的谷底（导数为 $0$），此时算法**收敛 (Convergence)**，抵达全局最优解。
    

ok，这一步就是**梯度下降算法**所做的事情。

为了找到最陡峭的方向，我们需要对代价函数 $J(w,b)$求偏导数（这就好比机器伸出脚去感知 $w$ 方向和 $b$ 方向的倾斜度）。

根据微积分法则，均方误差求导后的结果是这样的（你暂时不需要死记硬背怎么求导，只需要理解它的对应关系）：

1. **探寻坡度（算梯度）**：
    
    - $w$ 方向的坡度：
        
        $$dj\_dw = \frac{1}{m} \sum_{i=1}^{m} (h_{w,b}(x^{(i)}) - y^{(i)}) \cdot x^{(i)}$$
        
    - $b$ 方向的坡度：
        
        $$dj\_db = \frac{1}{m} \sum_{i=1}^{m} (h_{w,b}(x^{(i)}) - y^{(i)})$$
        
2. **迈出步伐（参数更新）**：
    
    - 同时朝着下坡方向迈步,其中学习率 $\alpha$，决定步伐的大小，
        
    - $w = w - \alpha \cdot dj\_dw$
        
    - $b = b - \alpha \cdot dj\_db$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ==========================================
# 1. 准备环境与数据 (经验 E)
# ==========================================
x_train = np.array([1.0, 1.5, 2.0, 2.5, 3.0]) # 面积
y_train = np.array([1.5, 2.5, 3.5, 4.0, 5.0]) # 价格
m = len(x_train)

# ==========================================
# 2. 蒙眼空投 (初始化参数)
# ==========================================
w = 0.0
b = 0.0

# 设置超参数
alpha = 0.1       # 学习率：决定迈步的大小
iterations = 1000 # 打算走多少步

# 用一个列表记录每一步的“海拔高度”(代价 J)，方便最后画图
J_history = []

# ==========================================
# 3. 梯度下降主循环 (学习算法启动)
# ==========================================
print("🚀 机器开始下山...\n")
for i in range(iterations):
    # 计算当前模型的预测值 h(x)
    h = w * x_train + b
    
    # 计算当前误差 (预测 - 真实)
    error = h - y_train
    
    # --- 动作 1：探寻坡度 (计算偏导数) ---
    dj_dw = (1 / m) * np.sum(error * x_train)
    dj_db = (1 / m) * np.sum(error)
    
    # --- 动作 2：迈出步伐 (同时更新参数) ---
    w = w - alpha * dj_dw
    b = b - alpha * dj_db
    
    # --- 记录与评估 ---
    # 计算此时的海拔 J(w,b)，仅作记录，不参与求导
    cost = (1 / (2 * m)) * np.sum(error ** 2)
    J_history.append(cost)
    
    # 每走 200 步，向人类汇报一次位置
    if i % 200 == 0:
        print(f"走了 {i:4d} 步: 海拔 J(误差) = {cost:.4f} | 当前参数: w = {w:.4f}, b = {b:.4f}")

print(f"\n🎯 到达谷底！最终定型参数: w = {w:.4f}, b = {b:.4f}")

# ==========================================
# 4. 验证结果：绘制下山轨迹图
# ==========================================
plt.figure(figsize=(8, 5))
plt.plot(range(iterations), J_history, color='purple', linewidth=2)
plt.title("Learning Curve: Cost J decreasing over time")
plt.xlabel("Iterations (Number of steps)")
plt.ylabel("Cost J (Altitude)")
plt.grid(True)
plt.show()


### 3.3 核心超参数：学习率 $\alpha$ (Learning Rate)

在上述过程中，有一个决定生死的参数：**你每次下山时，迈出的步长有多大？**
这个步长由**学习率 $\alpha$** 控制，它是机器学习调参的核心。

学习率 $\alpha$ 是我们在机器学习中遇到的**第一个，也是最重要的一个人工设置的参数（超参数 Hyperparameter）**。如果把它调坏了，再完美的模型和数据也会瞬间崩溃。

在梯度下降的参数更新公式中：

$$w = w - \alpha \frac{\partial J(w,b)}{\partial w}$$

- **导数项 $\frac{\partial J}{\partial w}$**：决定了机器下山的**方向**（左还是右）。
    
- **学习率 $\alpha$**：决定了机器在这个方向上**迈出的步伐有多大**。

为了直观感受其威力，我们将目标简化为一维二次函数 $J(w) = w^2$（真实谷底为 $w=0$），起点设为 $w=5$。

Python


In [ ]:
# 目标函数: J(w) = w^2 | 导数(梯度): dJ_dw = 2 * w
def gradient_descent_sim(w_start, alpha, iterations):
    w = w_start
    print(f"起点: w = {w:.4f}")
    for i in range(iterations):
        derivative = 2 * w
        w = w - alpha * derivative  # 参数更新公式
        print(f"第 {i+1} 步: w = {w:.4f}")

print("--- 场景 A：学习率极小 (alpha = 0.01) ---")
gradient_descent_sim(w_start=5.0, alpha=0.01, iterations=5)
# 现象：收敛极其缓慢，犹如蜗牛爬行，耗费大量算力。

print("\n--- 场景 B：学习率适中 (alpha = 0.1) ---")
gradient_descent_sim(w_start=5.0, alpha=0.1, iterations=5)
# 现象：稳步逼近，快速向 0 收敛。

print("\n--- 场景 C：学习率过大 (alpha = 1.1) ---")
gradient_descent_sim(w_start=5.0, alpha=1.1, iterations=5)
# 现象：由于步子太大，直接越过谷底跳到对面的山坡，且越跳越高，导致误差爆炸性发散 (Divergence)。


| **学习率 α 的设置**       | **物理直觉映射**                            | **算法表现与后果**                                                         |
| ------------------- | ------------------------------------- | ------------------------------------------------------------------- |
| **过小 (Too Small)**  | 像蚂蚁下山。虽然方向是对的，但每次只往下挪一毫米。             | **收敛极慢 (Slow Convergence)**：代价 $J$ 在下降，但可能需要迭代数百万次才能走到谷底，极其浪费算力和时间。 |
| **适中 (Just Right)** | 步伐稳健。一步步平稳地滑向谷底。                      | **完美收敛 (Converge)**：平稳且快速地找到使误差最小的最优解 $w$。                          |
| **过大 (Too Large)**  | 像绿巨人，一步跨过整个山谷。不仅跳到了对面的山坡上，而且落脚点比原来还高！ | **发散 (Diverge)**：机器在山谷两边反复横跳，且越跳越高。误差 $J$ 会越来越大，最终导致程序溢出崩溃。         |

**快到谷底时，需要手动减小 $\alpha$ 吗？**

吴恩达在课程中提出了一个极具启发性的问题：**如果我设置了一个固定的学习率 $\alpha$（比如 0.1），当机器越来越接近谷底时，它会不会因为步子还是那么大，从而跨过谷底走偏了？**

**答案是：不会！不需要手动减小 $\alpha$。**

- **底层逻辑**：机器迈出的**实际物理步长**，不仅仅取决于 $\alpha$，而是取决于 **$\alpha \times$ 导数（坡度）**。
    
- **物理映射**：随着机器越来越接近谷底，地形变得越来越平缓（导数趋近于 $0$）。因此，即使 $\alpha$ 保持不变，它乘上一个极小的导数后，**迈出的实际步长会自动变小**。
    
- **结论**：梯度下降自带“刹车系统”。只要初始的 $\alpha$ 不是大得离谱，机器会自动用越来越小的碎步稳稳停在谷底。

---
## 4. 全局总结与术语库 (Summary & Glossary)

### 4.1 笔记综述 (Overview)

单变量线性回归向我们展示了机器学习的终极工作流：

1. **输入经验**：准备包含特征 x 和目标 y 的训练数据。
    
2. **构建假设**：定义模型架构 h_{w,b}(x) = wx + b。
    
3. **量化误差**：使用均方误差代价函数 J(w,b) 评估当前模型的好坏。
    
4. **机器自学**：利用梯度下降算法，依托学习率 \alpha 不断迭代更新参数 w 和 b。
    
5. **模型收敛**：当 J 降至最低点时，训练结束，机器具备了预测未知数据的能力。
    

### 4.2 核心术语字典 (Glossary)

- **特征 (Feature, x)**：用来做预测的输入属性（如房屋面积）。
    
- **目标 (Target, y)**：需要预测的真实结果（如真实房价）。
    
- **假设函数 (Hypothesis, h_\theta 或 h_{w,b})**：机器从数据中学习到的规则/公式，用于输出预测值 \hat{y}。
    
- **参数 (Parameters, w, b)**：模型内部需要通过学习来不断调整的变量。
    
- **代价函数 (Cost Function, J)**：用于衡量模型整体预测误差的数学公式。
    
- **梯度下降 (Gradient Descent)**：一种寻找代价函数最小值的优化算法。
    
- **超参数 (Hyperparameter, \alpha)**：由工程师在训练开始前手动设置的参数（如学习率、迭代次数），机器无法自行学习。
    
- **收敛 (Convergence)**：算法找到了最优解，参数不再发生显著变化的状态。
    

---

## 5. 🛠️ 助教的工程视角 (Engineering Reality)

在学术界，我们用 `for` 循环手写梯度下降是为了理解底层微积分。但在 **Google** 或 **Meta** 的现代工业界，针对这种简单的线性模型，工程师会采用更高效的降维打击：

1. **向量化 (Vectorization)**：彻底抛弃低效的 `for` 循环。利用 NumPy 将数据转换为矩阵，依托底层的 C 语言和 GPU 进行并行矩阵乘法，速度可提升数十倍。
    
2. **正规方程 (Normal Equation)**：对于单变量或特征较少的小型数据集，工程师甚至不用梯度下降。通过线性代数公式 w = (X^T X)^{-1} X^T y 一步直接算出谷底坐标，无需设置 \alpha，无需迭代。
    
3. **调用高级框架**：在实际业务上线时，通常只需三行代码调用 `Scikit-Learn`（如 `LinearRegression().fit(X, y)`），底层已经做好了极其完善的异常处理和算力优化。


## 6.调用库代码

In [10]:
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# ==========================================
# 1. 准备数据 (与手写版有一处关键不同！)
# ==========================================
# 注意：sklearn 要求特征 X 必须是二维矩阵 (样本数, 特征数)
# 所以我们要用 .reshape(-1, 1) 把一维数组变成一列
x_train = np.array([1.0, 1.5, 2.0, 2.5, 3.0]).reshape(-1, 1) 
y_train = np.array([1.5, 2.5, 3.5, 4.0, 5.0])

# ==========================================
# 2. 核心 3 行：初始化与训练
# ==========================================
# 第 1 行：造一把“直尺”（实例化模型对象）
model = LinearRegression()

# 第 2 行：把数据喂给模型，让它自己找最佳位置（训练/拟合）
model.fit(x_train, y_train)

# 第 3 行：用训练好的模型进行预测 (比如预测 3000 平尺的房子)
x_new = np.array([[3.0]])
predicted_price = model.predict(x_new)

# ==========================================
# 3. 提取学到的知识 (查看底层参数)
# ==========================================
w = model.coef_[0]      # 提取权重 w (斜率)
b = model.intercept_    # 提取偏置 b (截距)

print(f"🤖 API 训练完成！")
print(f"学到的最佳参数: w = {w:.4f}, b = {b:.4f}")
print(f"预测 3000 平尺的房价为: {predicted_price[0]:.4f} (10万美元)")

🤖 API 训练完成！
学到的最佳参数: w = 1.7000, b = -0.1000
预测 3000 平尺的房价为: 5.0000 (10万美元)
